# Загрузка Pandas и очистка данных

In [1198]:
import pandas as pd
import numpy as np
import re
import math

df = pd.read_csv('main_task.csv')

## Restaurant_id

In [1199]:
# Selecting number from ID
df['Restaurant_id'] = df['Restaurant_id'].apply(lambda x: re.findall(r'\d+', str(x))[0])

In [1200]:
# Changing the type of data in the column
df['Restaurant_id'] = df['Restaurant_id'].astype(int)

## Price Range

In [1201]:
# Замена значений Price Range на числовые:
dict_price_range = {'$': 1, '$$ - $$$': 2, '$$$$': 3}
df['Price Range'] = df['Price Range'].replace(to_replace=dict_price_range)

In [1202]:
# Замена пустых значений Price Range на наиболее популярные значения:     
df['Price Range'] = df['Price Range'].\
fillna(df['Price Range'].value_counts().index[0])

In [1203]:
# Dummy- Creation
price = pd.get_dummies(df['Price Range'], drop_first=True)

In [1204]:
# Concatination in a DataFrame
df = pd.concat([df, price], axis=1)

## City

In [1205]:
# Dummy- Creation
city = pd.get_dummies(df['City'], drop_first=True)

In [1206]:
# Concatination in a DataFrame
df = pd.concat([df, city], axis=1)

## Cuisine Style

In [1207]:
# Замена пустых значений на "Uknown":
df['Cuisine Style'] = df['Cuisine Style'].fillna("Uknown")

In [1208]:
# Применение функции форматирования значений столбца:
def list_cuisine(cell):
    """Функция форматирует значения в ячейках столбца"""
    cell = str(cell)
    cell = cell.replace("[", "")
    cell = cell.replace("]", "")
    cell = cell.replace("'", "")
    cell = cell.split(', ')
    return cell


df['Cuisine Style'] = df['Cuisine Style'].apply(list_cuisine)

In [1209]:
# TASK -- Какое среднее количество кухонь предлагается в одном ресторане?
round(df['Cuisine Style'].apply(lambda x: len(x)).mean(), 1)

2.6

In [1210]:
# TASK -- Сколько типов кухонь представлено в наборе данных (без "Uknown")?
df = df.explode('Cuisine Style')
len(df['Cuisine Style'].value_counts())-1

125

In [1211]:
# TASK -- Какая кухня представлена в наибольшем количестве ресторанов?
df = df.explode('Cuisine Style')
df['Cuisine Style'].value_counts().index[0]

'Vegetarian Friendly'

In [1212]:
# Dummy- Creation:
cuisine = pd.get_dummies(df['Cuisine Style'], drop_first=True)

In [1213]:
# Concatination in a DataFrame
df = pd.concat([df, cuisine], axis=1)

## Reviews

In [1214]:
# Remove empty cells
df = df[df['Reviews'] != '[[], []]']

In [1215]:
# Find only data's numbers
df['Reviews'] = df['Reviews'].apply(lambda x: re.findall(r'\d+/\d+/\d+', str(x)))

In [1216]:
# TASK -- The latest review
# df = df.explode('Reviews')
# df['Reviews'] = pd.to_datetime(df['Reviews'])
# df['Reviews'].max()

In [1217]:
# New columns for 1st and 2nd review
df['1-Review'] = df['Reviews'].apply(lambda x: x[0])
df['2-Review'] = df['Reviews'].apply(lambda x: x[-1])

In [1218]:
# Conversion to date
df['1-Review'] = pd.to_datetime(df['1-Review'])
df['2-Review'] = pd.to_datetime(df['2-Review'])

In [1219]:
# Number of dates between reviews
df['Delta-Review'] = (abs(df['2-Review'] - df['1-Review'])).dt.days

In [1220]:
# TASK -- Max number of dates between reviews
df['Delta-Review'].sort_values(ascending=False).iloc[0]

3207

## Number of Reviews

In [1221]:
# Filling in the gaps with average number of reviews
df['Number of Reviews'] = df['Number of Reviews'].fillna(round(df['Number of Reviews'].mean()))

# Changing the type of data in the column
df['Number of Reviews'] = df['Number of Reviews'].astype(int)

## Rest Unused Columns

In [1222]:
# Deletion
df = df.drop(['City', 'Price Range', 'Cuisine Style', 'Reviews',\
              'URL_TA', 'ID_TA', '1-Review', '2-Review'], axis=1)

## Feature Engineering

In [1223]:
df['Rating/Ranking'] = df['Rating'] / df['Ranking']

## Normalization

In [1224]:
df['Ranking'] = np.log(df['Ranking'])
df['Rating'] = np.log(df['Rating'])
df['Number of Reviews'] = np.log(df['Number of Reviews'])
df['Rating/Ranking'] = np.log(df['Rating/Ranking'])

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [1225]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [1226]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [1227]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [1228]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [1229]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [1230]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.002134138404579139


In [1231]:
df

,Restaurant_id,Ranking,Rating,Number of Reviews,2.0,3.0,Athens,Barcelona,Berlin,Bratislava,...,Vegan Options,Vegetarian Friendly,Venezuelan,Vietnamese,Welsh,Wine Bar,Xinjiang,Yunnan,Delta-Review,Rating/Ranking
0,5569,8.625150,1.252763,5.267858,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,41,-7.372387
0,5569,8.625150,1.252763,5.267858,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,41,-7.372387
0,5569,8.625150,1.252763,5.267858,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,41,-7.372387
1,1535,7.337588,1.386294,2.302585,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,382,-5.951293
2,352,5.866468,1.504077,6.533789,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,2,-4.362391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39998,640,6.463029,1.386294,4.248495,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,23,-5.076735
39998,640,6.463029,1.386294,4.248495,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,23,-5.076735
39998,640,6.463029,1.386294,4.248495,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,23,-5.076735
39998,640,6.463029,1.386294,4.248495,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,23,-5.076735
